In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import open3d as o3d
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from src import Config
from src.common.logger import Logger
from src.feature import LasFileHandler, Voxelizer, VoxelFeatureExtractor
from src.feature.voxelizer.voxel_dto import VoxelDto
from src.feature.point_cloud_file import FileType, Filename, IFileHandler, DirectoryType

C:\Users\jatha\anaconda3\envs\venv\lib\site-packages\pyntcloud\utils\numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_count(xyz, indices, out):
C:\Users\jatha\anaconda3\envs\venv\lib\site-packages\pyntcloud\utils\numba.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_sum(xyz, indices, N, out):
C:\Users\jatha\anaconda3\envs\ve

In [4]:
logger: Logger = Logger()
point_cloud_file_handler: IFileHandler = LasFileHandler(logger=logger)

## Creating dataset for entire file

In [5]:
def create_dataset(*file_names: Filename, has_label: bool) -> None:
    datasets: list[pd.DataFrame] = []
    if has_label:
        directory_type: DirectoryType = DirectoryType.CROPPED
    else:
        directory_type: DirectoryType = DirectoryType.RAW

    for file_name in file_names:
        point_cloud_file_handler.open(
            directory_type=directory_type,
            filename=file_name,
            file_type=FileType.LAS
        )

        point_cloud: o3d.geometry.PointCloud = point_cloud_file_handler.render_point_cloud_object()
        voxelizer: Voxelizer = Voxelizer(point_cloud=point_cloud, logger=logger)
        voxel_dtos: list[VoxelDto] = voxelizer.voxels
        voxel_feature_extractors: list[tuple[VoxelDto, VoxelFeatureExtractor]] = []

        for dto in voxel_dtos:
            voxel: o3d.geometry.PointCloud = dto.point_cloud
            voxel_feature_extractor: VoxelFeatureExtractor = VoxelFeatureExtractor(point_cloud=voxel, logger=logger)
            voxel_feature_extractors.append((dto, voxel_feature_extractor))

        feature_frame: pd.DataFrame = pd.DataFrame(columns=[
            "grid_x",
            "grid_y",
            "grid_z",

            "r",
            "g",
            "b",

            "max_bound_x",
            "max_bound_y",
            "max_bound_z",

            "min_bound_x",
            "min_bound_y",
            "min_bound_z",

            "l1",
            "l2",
            "l3",
            "planarity",
            "linearity",
            "scattering",
            "omnivariance",
            "sum_of_eigenvalues",
            "eigenentropy",
            "anisotropy",
            "change_of_curvature",
            "z_range",
        ])
        
        for vfe in voxel_feature_extractors:
            voxel_dto: VoxelDto = vfe[0]
            voxel_feature_extractor: VoxelFeatureExtractor = vfe[1]
            
            # Values to store in the dataset
            dataset_values = np.concatenate([
                voxel_dto.grid_index,
                voxel_dto.voxel_color,
                voxel_dto.bounding_box.get_max_bound(),
                voxel_dto.bounding_box.get_min_bound(),
                voxel_feature_extractor.features.array
            ])
            
            insertion_index: int = len(feature_frame)
            feature_frame.loc[insertion_index] = dataset_values

        if has_label:
            feature_frame["label"] = file_name.value

        datasets.append(feature_frame)
        
    dataset: pd.DataFrame = pd.concat(datasets, axis=0)
    dataset = dataset.sample(frac=1)

    filepath: str = os.path.join(Config.DATASETS_DIR.value, "raw_point_cloud.csv")
    if has_label:
        filepath: str = os.path.join(Config.DATASETS_DIR.value, "labled_dataset.csv")

    dataset.to_csv(path_or_buf=filepath, index=False)

### Training data

In [6]:
filenames: list[Filename] = [Filename.BUILDINGS, Filename.TERRAIN, Filename.RIVERS, Filename.TREES]
create_dataset(*filenames, has_label=True)

INFO:src.common.logger.logger:Loaded 1516824 points from buildings.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 78005 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 78005 voxels...


Extracting voxels...:   0%|          | 0/78005 [00:00<?, ?it/s]

INFO:src.common.logger.logger:Extracted 49611 voxels, 28394 voxels were insufficient.
INFO:src.common.logger.logger:Loaded 2133081 points from terrain.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 67706 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 67706 voxels...


Extracting voxels...:   0%|          | 0/67706 [00:00<?, ?it/s]

INFO:src.common.logger.logger:Extracted 57436 voxels, 10270 voxels were insufficient.
INFO:src.common.logger.logger:Loaded 213299 points from river.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 19082 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 19082 voxels...


Extracting voxels...:   0%|          | 0/19082 [00:00<?, ?it/s]

INFO:src.common.logger.logger:Extracted 10197 voxels, 8885 voxels were insufficient.
INFO:src.common.logger.logger:Loaded 718861 points from trees.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 55426 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 55426 voxels...


Extracting voxels...:   0%|          | 0/55426 [00:00<?, ?it/s]

INFO:src.common.logger.logger:Extracted 30506 voxels, 24920 voxels were insufficient.


### Test data

In [7]:
create_dataset(Filename.CROPPED_RAW_DATA, has_label=False)

INFO:src.common.logger.logger:Loaded 8538911 points from cropped_raw_data.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 450802 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 450802 voxels...


Extracting voxels...:   0%|          | 0/450802 [00:00<?, ?it/s]

INFO:src.common.logger.logger:Extracted 294854 voxels, 155948 voxels were insufficient.
